In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.tensorboard import SummaryWriter
import sys

In [15]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs

In [4]:
#create writer for tensorboard and save in runs folder
writer= SummaryWriter("runs/mnist")

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [22]:
input_size=784
hidden_size=100
num_classes=10
num_epochs=5
batch_size=100
learning_rate=0.01

In [8]:
train_dataset=torchvision.datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=True)
test_dataset=torchvision.datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor())

In [9]:
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

In [10]:
ex_img,ex_label=next(iter(train_loader))

In [12]:
img_grid=torchvision.utils.make_grid(ex_img)
writer.add_image('mnist_img',img_grid)
writer.close()

In [17]:
class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.l1=nn.Linear(input_size,hidden_size)
        self.relu=nn.ReLU()
        self.l2=nn.Linear(hidden_size,num_classes)
        
    def forward(self,x):
        out=self.l1(x)
        out=self.relu(out)
        out=self.l2(out)
        return out
    
model=NeuralNet(input_size,hidden_size,num_classes)
model=model.to(device)

In [18]:
criterion=nn.CrossEntropyLoss()# applies softmax for us
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [21]:
ex_img=ex_img.to(device)
writer.add_graph(model,ex_img.reshape(-1,28*28))

In [23]:
tot_steps=len(train_loader)
run_loss=0.0
run_acc=0.0

In [24]:
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        
        outputs=model(images)
        loss=criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        run_loss+=loss.item()
        _,pred=torch.max(outputs.data,1)
        run_acc+=(pred==labels).sum().item()
        
        if(i+1)%100==0:
            writer.add_scalar('train_loss',run_loss/100,epoch*tot_steps+i)
            writer.add_scalar('train_acc',run_acc/100,epoch*tot_steps+i)
            run_loss=0.0
            run_acc=0.0
    print(loss.item())

0.29876452684402466
0.11573320627212524
0.0806870087981224
0.2847401797771454
0.10483856499195099
